In [1]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

model_name = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

src_text = "오늘 날씨가 정말 좋네요."
tokenizer.src_lang = "ko_KR"
encoded = tokenizer(src_text, return_tensors="pt")
generated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"])
print(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

It's really good weather today.


In [2]:
# !pip install bert_score
!pip install evaluate
!pip install sacrebleu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 6.6 MB/s eta 0:00:00


In [3]:
!pip install --upgrade datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [4]:
import os
from datasets import load_dataset

# Define a specific cache directory (replace with a path appropriate for your environment)
cache_directory = "./datasets_cache"
os.makedirs(cache_directory, exist_ok=True) # Create the directory if it doesn't exist

# Specify the path to the dataset repository on the Hugging Face Hub
dataset_path = "msarmi9/korean-english-multitarget-ted-talks-task"

# Explicitly define the data files based on the repository structure
# Assuming the data files are named train.jsonl, validation.jsonl, test.jsonl
data_files = {
    "train": "train.json",
    "validation": "dev.json",
    "test": "test.json"
}

# Load the dataset with explicit data files and specify the cache directory
ds = load_dataset(dataset_path, data_files=data_files, cache_dir=cache_directory)

print(ds) # Print the dataset structure to verify

README.md:   0%|          | 0.00/2.69k [00:00<?, ?B/s]

train.json:   0%|          | 0.00/41.5M [00:00<?, ?B/s]

dev.json:   0%|          | 0.00/461k [00:00<?, ?B/s]

test.json:   0%|          | 0.00/448k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['korean', 'english'],
        num_rows: 166215
    })
    validation: Dataset({
        features: ['korean', 'english'],
        num_rows: 1958
    })
    test: Dataset({
        features: ['korean', 'english'],
        num_rows: 1982
    })
})


In [ ]:
from tqdm import tqdm
from nltk.translate.bleu_score import sentence_bleu

In [5]:
!pip install evaluate

In [6]:
import evaluate
bleuScorer = evaluate.load("bleu")
chrfScorer = evaluate.load("chrf")

In [7]:
import torch
from tqdm import tqdm
# 62%|██████▏   | 700/1121 [4:23:11<3:19:59, 28.50s/it]i = 1560, bleu_score = 323.55672843138944, chrf_score = 75054.65326399355

In [8]:
context_sentence_length = 3
total = len(ds['test']['korean'])
bleu_score = 323.55672843138944
chrf_score = 75054.65326399355
for i in tqdm(range(1561, len(ds['test']['korean']))):
    src_text = ds['test']['korean'][i-context_sentence_length: i]
    # print(src_text)
    tokenizer.src_lang = "ko_KR"
    encoded = tokenizer(src_text, return_tensors="pt", padding=True, truncation=True)
    generated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"])
    predictions = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    # print()
    # print(f"predictions = {predictions}")
    reference = ds['test']['english'][i-context_sentence_length: i]
    # print(f"reference = {reference[-1]}, predictions = {predictions[-1]}")
    bleu_score += bleuScorer.compute(predictions = [predictions[-1]], references = [reference[-1]])['bleu']
    chrf_score += chrfScorer.compute(predictions = [predictions[-1]], references = [reference[-1]], word_order = 2)['score']

    # print(ds['test']['english'][i])

    if(i%20 == 0):
        print(f"i = {i}, bleu_score = {bleu_score}, chrf_score = {chrf_score}")
    # break
print(f"final bleu score = {bleu_score}, avg bleu score = {bleu_score / total}")
print(f"final chrf score = {chrf_score}, avg chrf score = {chrf_score / total}")

  5%|▍         | 20/421 [07:45<1:34:03, 14.07s/it]

i = 1580, bleu_score = 329.5336153086701, chrf_score = 76225.31896844447


 10%|▉         | 40/421 [16:41<1:55:09, 18.14s/it]

i = 1600, bleu_score = 336.5303939243971, chrf_score = 77500.49294679037


 14%|█▍        | 60/421 [22:59<2:41:23, 26.82s/it]

...

i = 1960, bleu_score = 470.52453059613316, chrf_score = 99355.259256985


100%|█████████▉| 420/421 [3:12:20<00:17, 17.68s/it]

i = 1980, bleu_score = 479.5518214265521, chrf_score = 100608.04435624527


100%|██████████| 421/421 [3:12:33<00:00, 27.44s/it]

final bleu score = 480.5518214265521, avg bleu score = 0.2424580330103694
final chrf score = 100708.04435624527, avg chrf score = 50.81132409497743


In [ ]:
import torch
from tqdm import tqdm
#43%|████▎     | 858/1979 [5:28:55<3:49:56, 12.31s/it]i = 860, bleu_score = 159.64170539529593, chrf_score = 39634.381744243685

In [ ]:
context_sentence_length = 3
total = len(ds['test']['korean'])
bleu_score = 159.64170539529593
chrf_score = 39634.381744243685
for i in tqdm(range(861, len(ds['test']['korean']))):
    src_text = ds['test']['korean'][i-context_sentence_length: i]
    # print(src_text)
    tokenizer.src_lang = "ko_KR"
    encoded = tokenizer(src_text, return_tensors="pt", padding=True, truncation=True)
    generated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"])
    predictions = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    # print()
    # print(f"predictions = {predictions}")
    reference = ds['test']['english'][i-context_sentence_length: i]
    # print(f"reference = {reference[-1]}, predictions = {predictions[-1]}")
    bleu_score += bleuScorer.compute(predictions = [predictions[-1]], references = [reference[-1]])['bleu']
    chrf_score += chrfScorer.compute(predictions = [predictions[-1]], references = [reference[-1]], word_order = 2)['score']

    # print(ds['test']['english'][i])

    if(i%20 == 0):
        print(f"i = {i}, bleu_score = {bleu_score}, chrf_score = {chrf_score}")
    # break
print(f"final bleu score = {bleu_score}, avg bleu score = {bleu_score / total}")
print(f"final chrf score = {chrf_score}, avg chrf score = {chrf_score / total}")

  2%|▏         | 20/1121 [05:44<4:32:33, 14.85s/it]

i = 880, bleu_score = 163.16765139995775, chrf_score = 40606.38737236328


  4%|▎         | 40/1121 [11:05<4:15:37, 14.19s/it]

i = 900, bleu_score = 168.86632611050658, chrf_score = 41688.44470140001


  5%|▌         | 60/1121 [17:48<4:14:30, 14.39s/it]

...

i = 1540, bleu_score = 317.41686622007313, chrf_score = 73875.24060820846


 62%|██████▏   | 700/1121 [4:23:11<3:19:59, 28.50s/it]

i = 1560, bleu_score = 323.55672843138944, chrf_score = 75054.65326399355


 63%|██████▎   | 704/1121 [4:26:04<2:37:36, 22.68s/it]


KeyboardInterrupt: 

In [ ]:
context_sentence_length = 3
total = len(ds['test']['korean'])
bleu_score = 0
chrf_score = 0
for i in tqdm(range(context_sentence_length, len(ds['test']['korean']))):
    src_text = ds['test']['korean'][i-context_sentence_length: i]
    # print(src_text)
    tokenizer.src_lang = "ko_KR"
    encoded = tokenizer(src_text, return_tensors="pt", padding=True, truncation=True)
    generated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"])
    predictions = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    # print()
    # print(f"predictions = {predictions}")
    reference = ds['test']['english'][i-context_sentence_length: i]
    # print(f"reference = {reference[-1]}, predictions = {predictions[-1]}")
    bleu_score += bleuScorer.compute(predictions = [predictions[-1]], references = [reference[-1]])['bleu']
    chrf_score += chrfScorer.compute(predictions = [predictions[-1]], references = [reference[-1]], word_order = 2)['score']

    # print(ds['test']['english'][i])

    if(i%20 == 0):
        print(f"i = {i}, bleu_score = {bleu_score}, chrf_score = {chrf_score}")
    # break
print(f"final bleu score = {bleu_score}, avg bleu score = {bleu_score / total}")
print(f"final chrf score = {chrf_score}, avg chrf score = {chrf_score / total}")

  1%|          | 18/1979 [04:37<8:39:38, 15.90s/it]

i = 20, bleu_score = 5.961851194944208, chrf_score = 1062.4002993462657


  2%|▏         | 38/1979 [10:36<8:27:10, 15.68s/it]

i = 40, bleu_score = 12.344939025108115, chrf_score = 2168.689596206624


  3%|▎         | 58/1979 [16:51<9:04:06, 16.99s/it]

...

 42%|████▏     | 838/1979 [5:23:39<5:10:30, 16.33s/it]

i = 840, bleu_score = 156.67866523823568, chrf_score = 38712.304982391615


 43%|████▎     | 858/1979 [5:28:55<3:49:56, 12.31s/it]

i = 860, bleu_score = 159.64170539529593, chrf_score = 39634.381744243685


 44%|████▎     | 861/1979 [5:29:59<5:39:18, 18.21s/it]

In [ ]:
P/len(range(1641, len(ds['test']['korean']))) #tensor([0.8111])

tensor([0.8111])

In [ ]:
R/len(range(1641, len(ds['test']['korean']))) #tensor([0.8014])

tensor([0.8014])

In [ ]:
context_sentence_length = 3
weights = (0.25, 0.25, 0.25, 0.25)
total = len(ds['test']['korean'])
bleu_score = 0
P = 0
R = 0
F1 = 0
bertScorer = BERTScorer(model_type='bert-base-uncased')
for i in tqdm(range(context_sentence_length, len(ds['test']['korean']))):
    src_text = ds['test']['korean'][i-context_sentence_length: i]
    # print(src_text)
    tokenizer.src_lang = "ko_KR"
    encoded = tokenizer(src_text, return_tensors="pt", padding=True, truncation=True)
    generated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"])
    predictions = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    # print()
    # print(f"predictions = {predictions}")
    reference = ds['test']['english'][i-context_sentence_length: i]
    # print(f"reference = {reference[-1]}, predictions = {predictions[-1]}")
    # print(ds['test']['english'][i])
    cur_score = sentence_bleu(reference[-1], predictions[-1], weights = weights)
    pre, recall, cur_F1 = bertScorer.score([predictions[-1]], [reference[-1]])
    P += pre
    R += recall
    F1 += cur_F1
    # print(f"my_score = {my_score}, cur_score = {cur_score}")
    bleu_score += cur_score
    # print(score)
    if(i%20 == 0):
        print(f"i = {i}, score = {bleu_score}, F1 = {F1}")
    # break

  0%|          | 0/1979 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or u

i = 20, score = 2.6137044507468994e-230, F1 = tensor([14.2368])


  2%|▏         | 38/1979 [11:39<9:01:33, 16.74s/it]

i = 40, score = 5.440382459191315e-230, F1 = tensor([29.5760])


  3%|▎         | 58/1979 [18:26<9:47:40, 18.36s/it]

...

 82%|████████▏ | 1618/1979 [10:50:11<2:45:26, 27.50s/it]

i = 1620, score = 2.1646423148533512e-228, F1 = tensor([1189.1992])


 83%|████████▎ | 1638/1979 [11:02:43<4:35:43, 48.52s/it]

i = 1640, score = 2.188918706259495e-228, F1 = tensor([1205.3110])


 84%|████████▎ | 1657/1979 [11:16:10<5:34:40, 62.36s/it]

In [ ]:
score

NameError: name 'score' is not defined

In [ ]:
reference[-1]

"Reset the three minutes, that's just not fair."

In [ ]:
predictions[-1]

"Reset the three minutes, that's the rule."

In [ ]:
weights = (0.25, 0.25, 0.25, 0.25)
total = len(ds['test']['korean'])
score = 0
for i in tqdm(range(len(ds['test']['korean']))):
    src_text = ds['test']['korean'][i]
    tokenizer.src_lang = "ko_KR"
    encoded = tokenizer(src_text, return_tensors="pt")
    generated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"])
    predictions = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    # print(predictions)
    reference = ds['test']['english'][i]
    # print(reference)
    # print(ds['test']['english'][i])
    score += sentence_bleu(predictions, reference, weights = weights)
    # print(score)
    if(i%20 == 0):
        print(f"i = {i}, score = {score}")
    break

  0%|          | 0/1982 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or u

i = 0, score = 1.2769777614215993e-231


In [ ]:
weights = (0.25, 0.25, 0.25, 0.25)
total = len(ds['test']['korean'])
score = 0
for i in tqdm(range(len(ds['test']['korean']))):
    src_text = ds['test']['korean'][i]
    tokenizer.src_lang = "ko_KR"
    encoded = tokenizer(src_text, return_tensors="pt")
    generated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"])
    predictions = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    # print(predictions)
    reference = ds['test']['english'][i]
    # print(reference)
    # print(ds['test']['english'][i])
    score += sentence_bleu(reference, predictions, weights = weights)
    # print(score)
    if(i%20 == 0):
        print(f"i = {i}, score = {score}")
    break

  0%|          | 0/1982 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or u

i = 0, score = 1.4012813265516817e-231
